In [1]:
# Common data processing
import os
import textwrap
import pandas as pd
from dotenv import load_dotenv
import ollama

# Langchain
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import OllamaEmbeddings
# Warning control
import warnings
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

True

In [2]:
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from sentence_transformers import SentenceTransformer
import numpy as np

In [4]:
fake_text = """
Umut Can Gulsen, born in 1998, was a Turkish and naturalised-French physicist and chemist who conducted pioneering research on quantum physics.
He was the first man to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields. His husband, Pierre Gulsen, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
He was, in 200, the first man to become a professor at the University of Paris. 
"""

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris. 
"""
isbank_text = """
İşbank, officially Türkiye İş Bankası,[a] is a commercial bank in Turkey. Founded by the orders of Mustafa Kemal Atatürk in 1924, it is the first bank to go into operation in the Republic of Türkiye.
As of a late 2023 report, the bank operates with 1.066 branches domestically, giving it one of the largest branch networks in Turkey. İşbank also has 22 branches across 11 different countries. The bank provides services to 24.3 million customers as of 2023.[6]

History
Following the culmination of World War I in 1918 and the subsequent dissolution of the Ottoman Empire in 1922, Turkey was declared a republic with Mustafa Kemal Atatürk as the president. Atatürk acknowledged the government’s need for a national financial institution to rebuild Turkey's economy following the debacle of the war. İşbank was founded on 26 August 1924 at the First Economy Congress in İzmir.[7]
Atatürk appointed Celâl Bayar, his close aide and then the Minister of Exchange Construction and Settlement, as the president of the newly formed bank. İşbank began operations with two branches and 37 staff under the leadership of Celâl Bayar, its first general manager. The bank was established with a capital of 1,000,000 TL of which 250,000 TL was covered by Atatürk himself, and the rest by private investors.[8] In 1927, the capital was raised by 2 Million TL so it could be merged with the National Credit Bank as equal partners.[9]
Established to utilize the smallest savings and transfer them to development, İşbank played a major role in the establishment and development of savings awareness in Turkish society, bringing the first piggy banks to Turkey in 1928.[10] In the following years, İşbank began to expand by opening branches across the country. In 1932, İşbank opened branches in Hamburg, Germany and Alexandria, Egypt, becoming the first Turkish bank to open branches abroad. In the 1950s,[11] İşbank expanded its portfolio of subsidiaries. The bank's subsidiaries became the driving force of Turkish industry, investing in and financing a wide range of businesses, particularly in the manufacturing sector. Having accelerated branch expansion in the 1960s and 1970s, İşbank paid special attention to the development of its overseas branch network in the 1980s. In the 1980s, İşbank began to offer a wide range of services to its customers in line with its multichannel banking vision. In 1982, İşbank introduced the first Automated Teller Machines (ATM) to Turkey, and at the same time, the bank named its ATMs as "Bankamatik" and registered this name as a trademark. With the launch of the "Blue Line" in 1991[12] and the first internet branch in 1997, İşbank assumed a pioneering role in this field. Since those years, İşbank has been carrying out R&D activities and studies on the use of new technologies to increase the quality and diversity of its services in line with the wishes and expectations of its customers.
"""

scifi_text = """
The Chronomancer’s Emissary
Prologue

In the year 2589, humanity had not only conquered the stars but had fractured the very fabric of time itself. Known as the “Chronomancers,” a select group of scientists had harnessed temporal energy, allowing humanity to traverse time and alter its course. However, meddling with time had unforeseen consequences. The universe was fracturing, time was leaking, and events from other periods bled unpredictably into the present. To counteract the chaos, the Chronomancers established the Temporal Enforcement Agency, or TEA, which sent specialized agents known as Emissaries to repair the timelines and prevent collapse.

One such Emissary was Kaden Sato. Strong-willed and fiercely intelligent, Kaden had been chosen for his unparalleled understanding of quantum mechanics and his bravery in the face of uncharted danger. Little did he know that he would soon embark on a mission unlike any before, one that would change the fate of time itself.

Chapter 1: The Ripple

Kaden sat in the briefing room of the TEA headquarters, his face illuminated by the cold blue light of the holographic interface in front of him. The Chronomancer Council, an assembly of the most brilliant minds in temporal science, looked down upon him through the hologram, their faces obscured by shadows.

“Kaden,” began Dr. Myra Elara, a stoic yet compassionate woman who had been Kaden’s mentor for years. “We’ve detected an anomaly in the year 2024. It’s… different from previous incidents. It’s not just an object or a person out of place. It’s an entire event.”

“An event?” Kaden raised an eyebrow, intrigued.

“Precisely,” she replied. “It’s something we call a Ripple Event. Somehow, a monumental event from 2145 has slipped back in time and is altering the entire flow of history in 2024.”

“What’s the event?” Kaden asked, leaning forward.

“It’s the Fusion Accord – the day humanity achieved unlimited energy. If this event occurs prematurely, the energy infrastructure of 2024 won’t handle it, potentially leading to a cataclysmic overload and collapse of global civilization.”

Kaden’s heart pounded. He’d been trained for irregularities, but an entire event displacement was unprecedented.

“You’ll be sent back to 2024 to correct this,” Myra continued, “and you’ll be given access to advanced temporal suppression equipment. Your mission is to locate the epicenter of the Ripple, determine the cause, and neutralize it. Time is already destabilizing; the longer we wait, the greater the damage.”

Kaden nodded, feeling the weight of the task.

Chapter 2: Arrival in 2024

The temporal displacement was jarring, as it always was. Kaden found himself standing in a busy street in New York City, his body adjusting to the unfamiliar sounds, smells, and sights of an era that felt almost quaint to him. People brushed past him, unaware of the massive temporal energy fluctuation he had just unleashed upon arrival.

He took a deep breath, activating his Temporal Inhibitor, a device strapped to his wrist that helped conceal his presence from anyone actively looking for disruptions in the timeline. The Inhibitor also fed him real-time data about the surrounding area. Glancing down, he saw an alert flashing: Anomaly detected. Epicenter identified - East Village Laboratory, January 12, 2024.

Making his way to the lab, Kaden felt an unshakable sense of urgency. As he approached, he noticed a heavy military presence, far too significant for a standard laboratory. Were they aware of the anomaly?

He quickly activated his Temporal Cloak, allowing him to move unseen, and slipped into the facility. He found the lab filled with computers, scientists, and a device he instantly recognized – a rudimentary fusion reactor, decades ahead of its time.

“Impossible…” he whispered, approaching the reactor. His Inhibitor pinged with information: Temporal Signature: Year 2145.

Someone had brought the technology from the future. But who?

Chapter 3: The Conspiracy

Kaden’s investigation led him to a series of encrypted files on one of the lab computers. Decoding them, he discovered the existence of a clandestine organization known as “The Continuum,” a rogue faction of scientists and engineers who believed the TEA was restricting humanity’s potential by controlling the timeline.

The Continuum’s manifesto declared that “Time belongs to all of humanity, not just the chosen few,” and they had been manipulating events throughout history to accelerate technological advancement. It was their belief that humanity was ready for fusion power, regardless of the instability it would cause.

Kaden realized the stakes were higher than he had initially thought. This wasn’t just a single incident; it was part of a larger plan to rewrite human history.

Chapter 4: The Shadowed Ally

Kaden soon discovered that he was not the only Emissary in 2024. Rumors whispered of another agent who had gone rogue, possibly working with the Continuum. Her name was Elise Tanaka, and she was once the finest operative TEA had ever trained. She had disappeared years ago, presumed dead in an accident involving temporal energy overload.

Tracking her movements across 2024, Kaden found her in an abandoned building in Brooklyn. She looked at him with a wry smile, her face illuminated by the soft glow of a fusion core.

“So they finally sent you,” she said, her voice a strange mixture of bitterness and relief.

“Elise,” he whispered, “why did you join them?”

“They’re not wrong, Kaden,” she replied, her eyes flashing with conviction. “Time shouldn’t be controlled. It should be free, accessible to everyone. The TEA isn’t protecting history; they’re holding humanity back.”

“By destabilizing entire centuries? You’re playing with forces you barely understand.”

“Maybe,” she admitted. “But we’ve grown too cautious, too afraid of change.”

He realized she was not the villain he had imagined. She truly believed that by unleashing technological advances, she was giving humanity a chance to evolve.

But he couldn’t let her continue.

Chapter 5: The Showdown

The climax of their struggle took place in the heart of the Continuum’s secret headquarters – a hidden facility beneath the streets of Manhattan, where the stolen fusion reactor was kept. As Kaden fought his way through Elise’s allies, he saw visions of the future flickering around him – alternate timelines where humanity had risen to great heights and others where it had been reduced to ashes.

At the core of the facility, Kaden and Elise stood face to face, each determined to fight for their beliefs.

“You’re afraid of progress,” she accused him.

“No,” he replied. “I’m afraid of chaos.”

Their battle was fierce, with temporal energy crackling around them. Every strike from their temporal weapons rippled across the room, creating fractures in reality. Moments from different eras appeared, and then dissolved: ancient Rome, World War II, a distant Martian colony.

Finally, Kaden managed to disable Elise’s Temporal Inhibitor, leaving her vulnerable.

“It doesn’t have to end like this,” he pleaded. “Come back with me. We can find a way to use this technology responsibly.”

Elise’s face softened, but she shook her head. “You don’t understand. This isn’t just a mission for me. It’s my life’s work.”

Before he could respond, she activated the fusion reactor. It surged with energy, dangerously close to causing a catastrophic time rift.

Kaden knew he had one choice: he activated his Inhibitor’s failsafe, sending an overload pulse to disrupt the fusion core. It would send both him and Elise back to their original timeline, but would hopefully stop the rift.

As the energy surrounded them, he saw Elise’s face one last time, her expression unreadable.

Epilogue

Kaden awoke in the infirmary of TEA headquarters, his mind reeling from the mission. He had succeeded in preventing the catastrophic fusion event, but at a great cost. Elise was gone, and the Continuum had fractured.

In the days that followed, he often thought of her – of her passion and conviction, and of the blurred line between hero and villain in a world where time itself was a battlefield. He knew the TEA would continue its work, but he also knew that Elise’s vision of a free timeline would always haunt him.

As he stared out into the sprawling city beyond TEA headquarters, he wondered if one day humanity would be ready to take charge of its own destiny… and if he would live to see it.
"""
# Convert the text into documents
documents = [Document(page_content=scifi_text)]

In [5]:
for doc in documents:
    if "\n" in doc.page_content:
        doc.page_content = doc.page_content.replace("\n\n", "\n")

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)

In [8]:
split_text = text_splitter.split_text(documents[0].page_content)
split_documents = []
for chunk in split_text:
    split_documents.append(Document(page_content=chunk))

In [9]:
# Initialize Ollama embeddings with the specified model
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

# Create FAISS vector store from documents
vector_store = FAISS.from_documents(split_documents, embedding_model)

In [10]:
# Load the local Ollama model
model_name = "llama3"  # Replace with the local model you want to use in Ollama

In [11]:
def retrieve_documents(query, vector_store, k=1):
    # Retrieve top-k relevant documents
    retrieved_docs = vector_store.similarity_search(query, k=k)
    return [doc.page_content for doc in retrieved_docs]

In [12]:
def generate_response(query, retrieved_docs, model_name):
    # Combine the query and retrieved documents into a prompt
    prompt = f"""Answer the question based only on the following context: 
    {retrieved_docs}
    Question: {query}
    Answer:"""
    
    # Use Ollama's `generate` function to get the response
    response = ollama.generate(model=model_name, prompt=prompt)
    return response['response']  # Adjust if necessary based on Ollama's response format

In [16]:
# Example query
query = "What is the story about?"

# Step 1: Retrieve relevant documents
retrieved_docs = retrieve_documents(query, vector_store, k=1)

# Step 2: Generate a response using Ollama
response = generate_response(query, retrieved_docs, model_name)
print(textwrap.fill(response, 60))

Based on the given context, it appears that the story is
about a significant event or series of events, rather than a
specific object or individual. The phrase "It's not just an
object or a person out of place" suggests that the story may
involve a larger-scale disruption or disturbance, and the
quote "Incidents. It's not just an object or a person out of
place. It's an entire event” implies that the story is about
a significant happening rather than a small occurrence.


In [36]:
# Example query
query = "What is the importance of Isbank for Turkey? List the key points."

# Step 1: Retrieve relevant documents
retrieved_docs = retrieve_documents(query, vector_store, k=1)

# Step 2: Generate a response using Ollama
response = generate_response(query, retrieved_docs, model_name)
print(response)

Based on the given context, here are the key points that highlight the importance of İşbank for Turkey:

1. **First bank in Republic of Türkiye**: İşbank was founded in 1924 by Mustafa Kemal Atatürk's orders, making it the first bank to operate in the newly established Republic of Turkey.
2. **National financial institution**: İşbank was established to rebuild Turkey's economy after World War I and the dissolution of the Ottoman Empire.
3. **Pioneering role in savings awareness**: İşbank played a major role in promoting savings culture in Turkish society, introducing the first piggy banks to Turkey in 1928.
4. **Branch network expansion**: İşbank expanded its branch network across the country and internationally, making it one of the largest branch networks in Turkey (1,066 domestic branches) and abroad (22 branches in 11 countries).
5. **Driving force of Turkish industry**: İŞbank's subsidiaries became a driving force for Turkish industry, investing and financing various businesses, p